In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

df = pd.read_csv("qoute_dataset.csv")

In [ ]:

quotes = df['quote']
quotes.head()

In [ ]:
quotes = quotes.str.lower()

In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)
quotes = quotes.apply(lambda x: x.translate(translator))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
vocab_size = 10000

tokinizer = Tokenizer(num_words=vocab_size)
tokinizer.fit_on_texts(quotes)

In [ ]:
word_index = tokinizer.word_index
print(len(word_index))
list(word_index.items())[:10]

In [ ]:
sequence = tokinizer.texts_to_sequences(quotes)

In [ ]:
for i in range(3):
  print(quotes[i])


In [ ]:
for i in range(3):
  print(sequence[i])

In [ ]:
X = []
y = []

for seq in sequence:
  for i in range(1,len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

In [ ]:
max_len = max(len(x) for x in X)
print(max_len)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')

In [ ]:
y = np.array(y)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense

In [ ]:
embedding_dim = 50
rnn_units = 128

In [ ]:
rnn_model = Sequential()

rnn_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
rnn_model.add(SimpleRNN(units=rnn_units))
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

In [ ]:
rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
rnn_model.summary()

In [ ]:
lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

In [ ]:
lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
lstm_model.summary()

In [ ]:
import pickle
with open("tokenizer.pkl", "wb") as f:
  pickle.dump(tokinizer, f)

In [ ]:
with open("max_len.pkl", "wb") as f:
  pickle.dump(max_len, f)

In [ ]:
lstm_model.fit(X_padded, y_one_hot, epochs=50, verbose=1)

In [ ]:

lstm_model.save("lstm_model.h5")